## AN EXAMPLE SIMULATION

This is an example simulation to test out the functioning of the simulation library

In [1]:
# IMPORT MODULES

import numpy as np
import matplotlib.pyplot as plt

from src.ringtrap_python.molecular_dynamics import sim_leapfrog
from src.ringtrap_python.potentials import LocalHarmonicPotential1D, InverseSquarePotential1D
from src.ringtrap_python.eigenmodes import get_linear_eq_pos

### 1 ATOM 1D HARMONIC POTENTIAL

Here we implement a single atom in a 1D harmonic potential and compare with theory

### 2 IDENTICAL ATOM 1D HARMONIC POTENTIAL

Here we compare 2 identical atoms in a harmonic potential and compare with theory

### 2 IDENTICAL CHARGES COLOUMB POTENTIAL

Here we compare 2 identical charged atoms for their coloumb interaction

### 2 CHARGED PARTICLES DIFFERENT MASS

Here we compare 2 charges of different masses for their coloumbic interactions